# New cut implementation


In [ ]:
# Coming soon
# seq = Dseq('aaGAATTCaa', circular=True)

# print('EcORI', EcoRI.ovhg, len(seq))
# for shift in range(len(seq)):
#     seq_shifted = seq.shifted(shift)
#     cut_site = seq_shifted.get_cutsites(EcoRI)[0][0]
#     print(shift, seq_shifted, cut_site, cut_site[0] - cut_site[1])

# seq = Dseq('ccTTAATTAAcc', circular=True)
# print('PacI', PacI.ovhg, len(seq))
# for shift in range(len(seq)):
#     seq_shifted = seq.shifted(shift)
#     cut_site = seq_shifted.get_cutsites(PacI)[0][0]
#     print(shift, seq_shifted, cut_site, cut_site[0] - cut_site[1])


# seq = Dseq('TTAAccccTTAA', circular=True)
# custom_cut = ((1, 11), type('DynamicClass', (), {'ovhg': 2})())
# print(seq.apply_cut(custom_cut, custom_cut).__repr__())

# print()

# custom_cut = ((1, 11), type('DynamicClass', (), {'ovhg': -10})())
# print(seq.apply_cut(custom_cut, custom_cut).__repr__())